<a href="https://colab.research.google.com/github/Stevenn9981/tic_tac_toe/blob/master/tic_tac_toe_part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

In [1]:
!sudo apt-get update
!sudo apt-get install -y xvfb ffmpeg freeglut3-dev
!pip install imageio
!pip install pyvirtualdisplay
!pip install tf-agents[reverb]
!pip install pyglet

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,279 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,455 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/universe

In [22]:
import base64
import imageio
import IPython
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import PIL.Image
import pyvirtualdisplay
import reverb
import random
import os
import shutil
import tempfile
import zipfile

import pygame as pg
from pygame import gfxdraw
from pygame.locals import *

import tensorflow as tf

from tf_agents.agents.categorical_dqn import categorical_dqn_agent
from tf_agents.agents.dqn import dqn_agent
from tf_agents.drivers import py_driver
from tf_agents.drivers import dynamic_step_driver
from tf_agents.environments import suite_gym
from tf_agents.environments import py_environment
from tf_agents.environments import tf_environment
from tf_agents.environments import tf_py_environment
from tf_agents.eval import metric_utils
from tf_agents.metrics import tf_metrics
from tf_agents.networks import sequential
from tf_agents.networks import q_network
from tf_agents.networks import categorical_q_network
from tf_agents.policies import policy_saver
from tf_agents.policies import py_tf_eager_policy
from tf_agents.policies import random_tf_policy
from tf_agents.policies import random_py_policy
from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.replay_buffers import reverb_replay_buffer
from tf_agents.replay_buffers import reverb_utils
from tf_agents.trajectories import trajectory
from tf_agents.specs import tensor_spec
from tf_agents.specs import array_spec
from tf_agents.utils import common
from tf_agents.trajectories import time_step as ts

tempdir = "./"

In [23]:
tf.version.VERSION

'2.14.0'

## Hyper-parameters

In [24]:
num_iterations = 50000 # @param {type:"integer"}

initial_collect_steps = 100  # @param {type:"integer"}
collect_steps_per_iteration = 30 # @param {type:"integer"}
replay_buffer_max_length = 100000  # @param {type:"integer"}

batch_size = 256  # @param {type:"integer"}
learning_rate = 3e-3  # @param {type:"number"}
log_interval = 200  # @param {type:"integer"}

num_eval_episodes = 10  # @param {type:"integer"}
eval_interval = 1000  # @param {type:"integer"}

gamma = 0.99
n_step_update = 2  # @param {type:"integer"}
num_atoms = 51  # @param {type:"integer"}
min_q_value = -20  # @param {type:"integer"}
max_q_value = 20  # @param {type:"integer"}
fc_layer_params = (100,)

BOARD_SIZE = 9 # @param {type:"integer"}

## Environment

In [25]:
import gym
import numpy as np
import math
from tabulate import tabulate

from typing import Tuple, List


class TicTacToeEnv1(py_environment.PyEnvironment):
    """
    Implementation of a TicTacToe Environment based on OpenAI Gym standards
    This class is modified from https://github.com/MauroLuzzatto/OpenAI-Gym-TicTacToe-Environment
    I modified that to follow the instructions of Question 1.
    """

    def __init__(self, mode='rgb_array') -> None:
        """This class contains a TicTacToe environment for OpenAI Gym

        Args:
            mode (str): render mode: human or rgb_array, which returns a text or RGB array of a picture that shows the current board
        """
        self.n_actions = BOARD_SIZE * BOARD_SIZE         # 9 * 9 grids to drop
        # state: 9 * 9 * 2: the first 9 * 9 layer is the opponent's play history and the second 9 * 9 layer is own.
        self._observation_spec = {'state': array_spec.BoundedArraySpec(shape=(BOARD_SIZE, BOARD_SIZE, 2), dtype=np.int_, minimum=0, maximum=1),
                                  'legal_moves': array_spec.ArraySpec(shape=(self.n_actions,), dtype=np.bool_)}

        self._action_spec = array_spec.BoundedArraySpec(shape=(), dtype=np.int_, minimum=0, maximum=80)
        self.render_mode = mode
        self.colors = [1, 2]
        self.screen = None
        self.gamma = gamma
        self.fields_per_side = BOARD_SIZE
        self.reset()

    def observation_spec(self):
        return self._observation_spec

    def action_spec(self):
        return self._action_spec

    def _reset(self) -> Tuple[np.ndarray, dict]:
        """
        reset the board game and state
        """
        self.board: np.ndarray = np.zeros(
            (self.fields_per_side, self.fields_per_side), dtype=int
        )
        self.current_player = 1
        self.info = {"players": {1: {"actions": []}, 2: {"actions": []}}, "Occupied": set(), "legal_moves": np.ones((self.n_actions,), dtype=bool)}

        # return self.decompose_board_to_state()
        observations_and_legal_moves = {'state': self.decompose_board_to_state(), 'legal_moves': self.info["legal_moves"]}
        return ts.restart(observations_and_legal_moves)

    def decompose_board_to_state(self):
        """
        Our state is a 9x9x2 matrix.
        The first layer is the opponent's play history, 0 means no stone, 1 means stones placed by the opponent.
        The second layer is the current player's history, 0 means no stone, 1 means stones placed by the current player.
        """
        opponent = 2 if self.current_player == 1 else 1
        o_plays = (self.board == opponent) * 1
        c_plays = (self.board == self.current_player) * 1
        return np.stack([o_plays, c_plays], axis=2)


    def _step(self, action: int) -> Tuple[np.ndarray, int, bool, dict]:
        """step function of the tictactoeEnv1

        Args:
          action (int): integer between [0, 80], each representing a field on the board

        Returns:
          state (np.array): state of 2 players' history, 0 means no stone, 1 means stones placed by the corresponding player (shape: 9x9x2).
          reward (int): reward of the currrent step
          done (boolean): true, if the game is finished
          (dict): empty dict for future game related information
        """
        action = int(action)
        if not (0 <= action < self.n_actions):
            raise ValueError(f"action '{action}' is not in action_space")

        reward = -0.1  # assign (negative) reward for every move done
        (row, col) = self.decode_action(action)

        # If the agent/player does not choose an empty square, randomly select an empty one.
        if self.board[row, col] != 0:
            print('WARNING: Not a legal step')
            action = random.choice(list(set(range(BOARD_SIZE * BOARD_SIZE)) - self.info["Occupied"]))
            (row, col) = self.decode_action(action)
            reward -= 1 # assign a negative reward if the play-out position is not empty


        # randomly select an adjacent position with probability 1/16
        if random.random() < 0.5:
            adjs = [[-1, -1], [-1, 0], [-1, 1], [0, 1], [0, -1], [1, -1], [1, 0], [1, 1]]
            adj = random.choice(adjs)
            row, col = row + adj[0], col + adj[1]

        if 0 <= row < BOARD_SIZE and 0 <= col < BOARD_SIZE and self.board[row, col] == 0:
            self.board[row, col] = self.current_player  # drop the piece on the field
            win = self._is_win(self.current_player, row, col)

            action = row * BOARD_SIZE + col
            self.info["players"][self.current_player]["actions"].append(action)
            self.info["Occupied"].add(action)
            self.info['legal_moves'][action] = False
        else:
            win = False

        if win:
            reward += 3

        done = (win or len(self.info["Occupied"]) == BOARD_SIZE * BOARD_SIZE)
        self.current_player = self.current_player + 1 if self.current_player == 1 else 1
        state = self.decompose_board_to_state()

        observations_and_legal_moves = {'state': state, 'legal_moves': self.info['legal_moves']}
        # return next_s, reward, done, self.info
        if done:
            return ts.termination(observations_and_legal_moves, reward)
        else:
            return ts.transition(observations_and_legal_moves, reward, self.gamma)

    def _is_win(self, color: int, r: int, c: int) -> bool:
        """check if this player results in a winner

        Args:
            color (int): of the player
            r (int): row of the current play
            c (int): column of the current play

        Returns:
            bool: indicating if there is a winner
        """

        # check if four equal stones are aligned (horizontal, verical or diagonal)
        directions = [[0, 1], [1, 0], [1, 1], [1, -1]]

        for direct in directions:
            count = 0
            for offset in range(-3, 4):
                if 0 <= r + offset * direct[0] < 9 and 0 <= c + offset * direct[1] < 9:
                    if self.board[r + offset * direct[0], c + offset * direct[1]] == color:
                        count += 1
                        if count == 4:
                            return True
                    else:
                        count = 0

        return False

    def decode_action(self, action: int) -> List[int]:
        """decode the action integer into a colum and row value

        0 = upper left corner
        8 = lower right corner

        Args:
            action (int): action

        Returns:
            List[int, int]: a list with the [row, col] values
        """
        col = action % BOARD_SIZE
        row = action // BOARD_SIZE
        assert 0 <= col < BOARD_SIZE
        return [row, col]

    def render(self) -> None:
        """render the board

        The following charachters are used to represent the fields,
            '-' no stone
            'O' for player 0
            'X' for player 1

        An example for a 3x3 game board:
            ╒═══╤═══╤═══╕
            │ O │ - │ - │
            ├───┼───┼───┤
            │ - │ X │ - │
            ├───┼───┼───┤
            │ - │ - │ - │
            ╘═══╧═══╧═══╛
        """
        board = np.zeros((BOARD_SIZE, BOARD_SIZE), dtype=str)
        for ii in range(BOARD_SIZE):
            for jj in range(BOARD_SIZE):
                if self.board[ii, jj] == 0:
                    board[ii, jj] = "-"
                elif self.board[ii, jj] == 1:
                    board[ii, jj] = "X"
                elif self.board[ii, jj] == 2:
                    board[ii, jj] = "O"

        if self.render_mode == "human":
            board = tabulate(board, tablefmt="fancy_grid")
            print(board)
            print("\n")


        width = height = 400

        white = (255, 255, 255)
        line_color = (0, 0, 0)

        os.environ["SDL_VIDEODRIVER"] = "dummy"
        pg.init()

        # Set up the drawing window
        if self.screen is None:
          self.screen = pg.display.set_mode([width + 16, height + 16])

        self.screen.fill(white)
        # drawing vertical lines
        for i in range(10):
          pg.draw.line(self.screen, line_color, (width / BOARD_SIZE * i, 0), (width / BOARD_SIZE * i, height), 2)

        # drawing horizontal lines
        for i in range(10):
          pg.draw.line(self.screen, line_color, (0, height / BOARD_SIZE * i), (width, height / BOARD_SIZE * i), 2)
        pg.display.flip()

        # drawing noughts and crosses
        for i in range(BOARD_SIZE):
          for j in range(BOARD_SIZE):
            if self.board[i, j] == 1: # Draw crosses
              pg.draw.lines(self.screen, line_color, True, [(width / BOARD_SIZE * (j + 0.5) - 10,
                                                        height / BOARD_SIZE * (i + 0.5) - 10),
                                                      (width / BOARD_SIZE * (j + 0.5) + 10,
                                                        height / BOARD_SIZE * (i + 0.5) + 10)], 3)
              pg.draw.lines(self.screen, line_color, True, [(width / BOARD_SIZE * (j + 0.5) - 10,
                                                        height / BOARD_SIZE * (i + 0.5) + 10),
                                                        (width / BOARD_SIZE * (j + 0.5) + 10,
                                                          height / BOARD_SIZE * (i + 0.5) - 10)], 3)
            elif self.board[i, j] == 2: # Draw noughts
              pg.draw.circle(self.screen, line_color, (width / BOARD_SIZE * (j + 0.5), height / BOARD_SIZE * (i + 0.5)), 12, 3)

        board = np.transpose(
                np.array(pg.surfarray.pixels3d(self.screen)), axes=(1, 0, 2)
            )

        return board

In [26]:
def observation_and_action_constraint_splitter(obs):
    return obs['state'], obs['legal_moves']

## Random play test

In [27]:
# py_env = suite_gym.wrap_env(TicTacToeEnv1())
py_env = TicTacToeEnv1()
tf_env = tf_py_environment.TFPyEnvironment(py_env)

In [28]:
def embed_mp4(filename):
    """Embeds an mp4 file in the notebook."""
    video = open(filename,'rb').read()
    b64 = base64.b64encode(video)
    tag = '''
    <video width="480" height="480" controls>
      <source src="data:video/mp4;base64,{0}" type="video/mp4">
    Your browser does not support the video tag.
    </video>'''.format(b64.decode())

    return IPython.display.HTML(tag)

def create_policy_eval_video(policy, filename, fps=2):
    py_env = TicTacToeEnv1()
    tf_env = tf_py_environment.TFPyEnvironment(py_env)
    filename = filename + ".mp4"
    with imageio.get_writer(filename, fps=fps) as video:
      time_step = tf_env.reset()
      video.append_data(py_env.render())
      while not time_step.is_last():
        action_step = policy.action(time_step)
        time_step = tf_env.step(action_step.action)
        video.append_data(py_env.render())

    return embed_mp4(filename)

In [29]:
random_policy = random_tf_policy.RandomTFPolicy(tf_env.time_step_spec(), tf_env.action_spec(),
    observation_and_action_constraint_splitter=observation_and_action_constraint_splitter)
create_policy_eval_video(random_policy, "random-agent")

## Agent

In [30]:
train_py_env = TicTacToeEnv1()
eval_py_env = TicTacToeEnv1()

train_env = tf_py_environment.TFPyEnvironment(train_py_env)
eval_env = tf_py_environment.TFPyEnvironment(eval_py_env)

In [31]:
train_env.time_step_spec()

TimeStep(
{'discount': BoundedTensorSpec(shape=(), dtype=tf.float32, name='discount', minimum=array(0., dtype=float32), maximum=array(1., dtype=float32)),
 'observation': {'legal_moves': TensorSpec(shape=(81,), dtype=tf.bool, name=None),
                 'state': BoundedTensorSpec(shape=(9, 9, 2), dtype=tf.int64, name=None, minimum=array(0), maximum=array(1))},
 'reward': TensorSpec(shape=(), dtype=tf.float32, name='reward'),
 'step_type': TensorSpec(shape=(), dtype=tf.int32, name='step_type')})

In [32]:
conv_layer_params = [16, 32]
action_tensor_spec = tensor_spec.from_spec(tf_env.action_spec())
num_actions = action_tensor_spec.maximum - action_tensor_spec.minimum + 1

# Define a helper function to create Conv layers configured with the right
# activation and kernel initializer.
def conv_layer(num_units):
  return tf.keras.layers.Conv2D(
                      filters=num_units,
                      kernel_size=[3, 3],
                      padding="same",
                      data_format="channels_last",
                      activation=tf.nn.leaky_relu,
                      dtype=float)


# QNetwork consists of a sequence of Conv layers followed by a dense layer
# with `num_actions` units to generate one q_value per available action as
# its output.
normalization1 = tf.keras.layers.BatchNormalization()
normalization2 = tf.keras.layers.BatchNormalization()
conv_layers = [conv_layer(num_units) for num_units in conv_layer_params]
action_conv = tf.keras.layers.Conv2D(filters=4,
                    kernel_size=[1, 1], padding="same",
                    data_format="channels_last",
                    activation=tf.nn.leaky_relu)
flatten = tf.keras.layers.Flatten()
q_values_layer = tf.keras.layers.Dense(
    num_actions,
    activation=None,
    kernel_initializer=tf.keras.initializers.RandomUniform(
        minval=-0.03, maxval=0.03),
    bias_initializer=tf.keras.initializers.Constant(-0.2))
q_net = sequential.Sequential([normalization1] + conv_layers + [action_conv, normalization2, flatten, q_values_layer])


In [33]:
class ConvNestedLayer(tf.keras.layers.Layer):
    def __init__(self, *args, **kwds):
        super().__init__(*args, **kwds)
        self.conv1 = tf.keras.layers.Conv2D(
                      filters=32,
                      kernel_size=[3, 3],
                      padding="same",
                      data_format="channels_last",
                      activation=tf.nn.leaky_relu,
                      dtype=float)
        self.conv2 = tf.keras.layers.Conv2D(
                      filters=64,
                      kernel_size=[3, 3],
                      padding="same",
                      data_format="channels_last",
                      activation=tf.nn.leaky_relu,
                      dtype=float)
        self.norm1 = tf.keras.layers.BatchNormalization()
        self.norm2 = tf.keras.layers.BatchNormalization()
        self.flatten = tf.keras.layers.Flatten()

    def call(self, x):
        x = self.norm1(x)
        x = self.conv1(x)
        x = self.norm2(x)
        x = self.conv2(x)
        x = self.flatten(x)
        return x

In [34]:
categorical_q_net = categorical_q_network.CategoricalQNetwork(
    train_env.observation_spec()['state'],
    train_env.action_spec(),
    num_atoms=num_atoms,
    preprocessing_layers = ConvNestedLayer(),
    fc_layer_params=fc_layer_params)

In [35]:
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

train_step_counter = tf.Variable(0)

# agent = dqn_agent.DqnAgent(
#     train_env.time_step_spec(),
#     train_env.action_spec(),
#     q_network=q_net,
#     optimizer=optimizer,
#     td_errors_loss_fn=common.element_wise_squared_loss,
#     train_step_counter=train_step_counter)


agent = categorical_dqn_agent.CategoricalDqnAgent(
    train_env.time_step_spec(),
    train_env.action_spec(),
    categorical_q_network=categorical_q_net,
    observation_and_action_constraint_splitter=observation_and_action_constraint_splitter,
    optimizer=optimizer,
    min_q_value=min_q_value,
    max_q_value=max_q_value,
    n_step_update=n_step_update,
    td_errors_loss_fn=common.element_wise_squared_loss,
    gamma=gamma,
    train_step_counter=train_step_counter)

agent.initialize()

## Policy

In [36]:
eval_policy = agent.policy
collect_policy = agent.collect_policy
random_policy = random_tf_policy.RandomTFPolicy(train_env.time_step_spec(), train_env.action_spec(),
    observation_and_action_constraint_splitter=observation_and_action_constraint_splitter)

In [37]:
def compute_avg_return(environment, policy, num_episodes=10):

  total_return = 0.0
  for _ in range(num_episodes):
    time_step = environment.reset()
    episode_return = 0.0
    while not time_step.is_last():
      action_step = policy.action(time_step)
      time_step = environment.step(action_step.action)
      episode_return += time_step.reward
    total_return += episode_return

  avg_return = total_return / num_episodes
  return avg_return.numpy()[0]


# See also the metrics module for standard implementations of different metrics.
# https://github.com/tensorflow/agents/tree/master/tf_agents/metrics

In [38]:
compute_avg_return(eval_env, agent.policy, 1)

0.10000062

## Replay Buffer

In [43]:
replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(
    data_spec=agent.collect_data_spec,
    batch_size=train_env.batch_size,
    max_length=replay_buffer_max_length)

def collect_step(environment, policy):
    time_step = environment.current_time_step()
    if time_step.is_last():
        time_step = environment.reset()
    action_step = policy.action(time_step)
    next_time_step = environment.step(action_step.action)
    traj = trajectory.from_transition(time_step, action_step, next_time_step)

    # Add trajectory to the replay buffer
    replay_buffer.add_batch(traj)

for _ in range(initial_collect_steps):
    collect_step(train_env, random_policy)

# Dataset generates trajectories with shape [BxTx...] where
# T = n_step_update + 1.
dataset = replay_buffer.as_dataset(
    num_parallel_calls=3, sample_batch_size=batch_size,
    num_steps=n_step_update + 1).prefetch(3)

iterator = iter(dataset)

## Set up the checkpointer and Policy saver

In [40]:
def create_zip_file(dirname, base_filename):
  return shutil.make_archive(base_filename, 'zip', dirname)

In [41]:
policy_dir = os.path.join(tempdir, 'policy')
tf_policy_saver = policy_saver.PolicySaver(agent.policy)

## Train the agent

In [ ]:
try:
  %%time
except:
  pass

# (Optional) Optimize by wrapping some of the code in a graph using TF function.
agent.train = common.function(agent.train)

# Reset the train step.
agent.train_step_counter.assign(0)

# Evaluate the agent's policy once before training.
avg_return = compute_avg_return(eval_env, agent.policy, num_eval_episodes)
returns = [avg_return]

# Reset the environment.
time_step = train_py_env.reset()

for _ in range(num_iterations):

    # Collect a few steps using collect_policy and save to the replay buffer.
    for _ in range(collect_steps_per_iteration):
      collect_step(train_env, agent.collect_policy)

    # Sample a batch of data from the buffer and update the agent's network.
    experience, unused_info = next(iterator)
    train_loss = agent.train(experience).loss

    step = agent.train_step_counter.numpy()

    if step % log_interval == 0:
      print('step = {0}: loss = {1}'.format(step, train_loss))

    if step % eval_interval == 0:
      avg_return = compute_avg_return(eval_env, agent.policy, num_eval_episodes)
      print('step = {0}: Average Return = {1}'.format(step, avg_return))
      tf_policy_saver.save(policy_dir)
      policy_zip_filename = create_zip_file(policy_dir, os.path.join(tempdir, 'exported_policy'))
      returns.append(avg_return)

step = 200: loss = 1.6713066101074219
step = 400: loss = 2.169821262359619
step = 600: loss = 2.179478406906128
step = 800: loss = 1.289841890335083
step = 1000: loss = 1.757983684539795


step = 1000: Average Return = -0.6199987530708313


/usr/local/lib/python3.10/dist-packages/tensorflow/python/saved_model/nested_structure_coder.py:458: UserWarning: Encoding a StructuredValue with type tfp.distributions.Deterministic_ACTTypeSpec; loading this StructuredValue will require that this type be imported and registered.
  warnings.warn("Encoding a StructuredValue with type %s; loading this "


step = 1200: loss = 2.1051955223083496
step = 1400: loss = 2.4827213287353516
step = 1600: loss = 2.1336159706115723
step = 1800: loss = 2.0858232975006104
step = 2000: loss = 1.8724911212921143
step = 2000: Average Return = -1.139998197555542


/usr/local/lib/python3.10/dist-packages/tensorflow/python/saved_model/nested_structure_coder.py:458: UserWarning: Encoding a StructuredValue with type tfp.distributions.Deterministic_ACTTypeSpec; loading this StructuredValue will require that this type be imported and registered.
  warnings.warn("Encoding a StructuredValue with type %s; loading this "


...

step = 19200: loss = 2.231112241744995
step = 19400: loss = 2.227442979812622
step = 19600: loss = 2.21697998046875
step = 19800: loss = 2.217949628829956
step = 20000: loss = 2.2224948406219482
step = 20000: Average Return = 0.3200004994869232


/usr/local/lib/python3.10/dist-packages/tensorflow/python/saved_model/nested_structure_coder.py:458: UserWarning: Encoding a StructuredValue with type tfp.distributions.Deterministic_ACTTypeSpec; loading this StructuredValue will require that this type be imported and registered.
  warnings.warn("Encoding a StructuredValue with type %s; loading this "


step = 20200: loss = 2.228635311126709
step = 20400: loss = 2.23043155670166
step = 20600: loss = 2.2261743545532227
step = 20800: loss = 2.216956615447998
step = 21000: loss = 2.223219871520996
step = 21000: Average Return = 0.17000070214271545


/usr/local/lib/python3.10/dist-packages/tensorflow/python/saved_model/nested_structure_coder.py:458: UserWarning: Encoding a StructuredValue with type tfp.distributions.Deterministic_ACTTypeSpec; loading this StructuredValue will require that this type be imported and registered.
  warnings.warn("Encoding a StructuredValue with type %s; loading this "


step = 21200: loss = 2.2269272804260254
step = 21400: loss = 2.226865291595459
step = 21600: loss = 2.2292799949645996
step = 21800: loss = 2.222665548324585


## Load the trained model

In [ ]:
saved_policy = tf.saved_model.load(policy_dir)
create_policy_eval_video(agent.policy, "trained-agent")

## TODO



*   Check whether all functions are correct
*   Check whether the environment is correctly implement
*   Add the variant in `step` function
*   Test whether the agent always chooses an *empty* square and whether they can go to an adjacent place correctly

